Covid Cases Stratified

Author: Zachary Stanke


In [1]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import CovidEM
import CovidCI
import warnings
import math
import sys
from tabulate import tabulate
import csv

import imp
imp.reload(CovidEM) #applies changes made to file

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

print(cases)
print(prem["can"])


     x00  x10   x20    x30    x40    x50    x60    x70  x80plus
can   43   53   250    301    315    380    300    163      139
chn  416  549  3619   7600   8571  10008   8583   3918     1408
gbr  701  822  7299  10048  12319  15441  12081  14655    24325
isr  319  732  1865   1206   1051   1134    969    495      247
ita   63  118   511    819   1523   2480   2421   2849     2533
nld   71  214  1973   2152   2839   4719   3805   4531     6219
[[ 2.87087681  0.94357477  0.37358052  0.25317159  0.33586201  0.65584323
   1.03423059  0.92004231  0.4489512   0.29422722  0.31204529  0.25033322
   0.16140108  0.13253666  0.07109389  0.03790854]
 [ 0.86595357  4.4816395   0.88978665  0.25207481  0.15642724  0.40257766
   0.76881514  0.93100172  0.74227614  0.31622672  0.21051978  0.17382395
   0.14895517  0.11229653  0.04629893  0.03547422]
 [ 0.21203609  1.41985066  6.67547793  0.80122709  0.29643418  0.25208034
   0.46080405  0.71878219  0.96370292  0.5479558   0.28606516  0.14325162
   0.0

In [2]:
from math import log2, isclose

# Testing
print(cases_dict["can"])
print(CovidEM.case_dim_split(cases_dict["can"]))

print(CovidEM._test_kl_div())

[43, 53, 250, 301, 315, 380, 300, 163, 139]
[21.5, 21.5, 26.5, 26.5, 125.0, 125.0, 150.5, 150.5, 157.5, 157.5, 190.0, 190.0, 150.0, 150.0, 81.5, 81.5]
Test executed successfully


In [3]:
# SCV Eig testing
imp.reload(CovidEM) #applies changes made to file

s = np.array([1]*7)
v = np.array([1]*7)
print("Canada")
print(CovidEM.scv_eig(s, prem["can"], v, debug = True))
print("Britain")
print(CovidEM.scv_eig(s, prem["gbr"], v))

Canada
[0.0351210567678103, 0.03819895170471739, 0.05529301475700961, 0.1468755047971597, 0.09332758935091987, 0.08013776268366546, 0.07329130027020123, 0.07462789985799026, 0.07645403197527066, 0.07490933153588901, 0.08101337961507346, 0.0648165295063615, 0.036744011906806255, 0.027534567403347615, 0.02301405127425486, 0.018641016593522925]
<class 'numpy.float64'>
[0.0351210567678103, 0.03819895170471739, 0.05529301475700961, 0.1468755047971597, 0.09332758935091987, 0.08013776268366546, 0.07329130027020123, 0.07462789985799026, 0.07645403197527066, 0.07490933153588901, 0.08101337961507346, 0.0648165295063615, 0.036744011906806255, 0.027534567403347615, 0.02301405127425486, 0.018641016593522925]
Britain
[0.03744260698802723, 0.04589034304202844, 0.06700906045434206, 0.1676439106566881, 0.08631109570001717, 0.08081346455570045, 0.0654897385967, 0.0693804022559647, 0.07179263541272551, 0.0816156674580504, 0.06055125123662846, 0.05669465696225351, 0.02748843401984161, 0.03085358381005995,

In [4]:
theta = np.array([1]*14)

print("Canada: ",CovidEM.Covid_KL_k(theta, prem["can"], cases_dict["can"]))
print("China: ",CovidEM.Covid_KL_k(theta, prem["chn"], cases_dict["chn"]))
print("Britain: ",CovidEM.Covid_KL_k(theta, prem["gbr"], cases_dict["gbr"]))
print("Isreal: ",CovidEM.Covid_KL_k(theta, prem["isr"], cases_dict["isr"]))
print("Italy: ",CovidEM.Covid_KL_k(theta, prem["ita"], cases_dict["ita"]))
print("Netherlands: ",CovidEM.Covid_KL_k(theta, prem["nld"], cases_dict["nld"]))

Canada:  0.2426314756176195
China:  0.4194522670332567
Britain:  0.4936969997127147
Isreal:  0.1804331829376997
Italy:  0.6715661066523932
Netherlands:  0.888667378830992


In [5]:
# the bounds
theta = np.array([1]*14)
xmin = [0] * 14
xmax = [100] * 14
my_bounds = [(low, high) for low, high in zip(xmin, xmax)]

print ("Canada:\n")
kwargs = dict(args = (prem["can"], cases_dict["can"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs))

print ("\n\nChina:\n")
kwargs = dict(args = (prem["chn"], cases_dict["chn"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nBritain:\n")
kwargs = dict(args = (prem["gbr"], cases_dict["gbr"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nIsreal:\n")
kwargs = dict(args = (prem["isr"], cases_dict["isr"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nItaly:\n")
kwargs = dict(args = (prem["ita"], cases_dict["ita"]), method="L-BFGS-B")
print(optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs).x)

print ("\n\nNetherlands:\n")
kwargs = dict(args = (prem["nld"], cases_dict["nld"]), method="L-BFGS-B")
res = optimize.basinhopping(CovidEM.Covid_KL_k, theta, minimizer_kwargs = kwargs)
print(res)
print(res.x)

Canada:



C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


                        fun: 0.002907398787928747
 lowest_optimization_result:       fun: 0.002907398787928747
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.75341165e-04, -4.75765781e-04, -6.42975174e-04, -1.55791006e-04,
        9.34154929e-04,  1.05553908e-03, -8.63716656e-04, -1.43755581e-04,
        1.48114854e-04, -6.01744349e-05, -1.17640273e-06,  2.98854693e-05,
        1.59140931e-04, -7.66691403e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 270
      nit: 16
     njev: 18
   status: 0
  success: True
        x: array([0.37592442, 0.43837916, 1.2281646 , 1.20034388, 1.0687901 ,
       1.72653418, 1.45259078, 3.89037548, 0.02948645, 0.20359417,
       1.28192343, 4.0017957 , 1.63703745, 1.50836453])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 55
                       nfev: 25965
                        nit: 100
                   

In [6]:
theta = np.array([1]*14)
# the bounds
#xmin = [0] * 14
#xmax = [100] * 14
#my_bounds = [(low, high) for low, high in zip(xmin, xmax)]

kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
res = optimize.basinhopping(CovidEM.Covid_KL, theta, minimizer_kwargs = kwargs)
print(res)

                        fun: 0.3245744260159009
 lowest_optimization_result:       fun: 0.3245744260159009
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0225256 ,  0.00598465,  0.01055354, -0.01139346,  0.00652332,
        0.01688392, -0.01205673, -0.00062759,  0.00163072,  0.00713293,
        0.01017609,  0.00571815,  0.00143089, -0.00821457])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 150
      nit: 8
     njev: 10
   status: 0
  success: True
        x: array([0.31154514, 0.40282449, 1.23018648, 1.5625491 , 1.60270277,
       1.9161011 , 2.04040767, 1.32679041, 1.23927424, 0.78829547,
       0.04571445, 2.29161589, 2.02793441, 2.38211475])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 62
                       nfev: 23940
                        nit: 100
                       njev: 1596
                          x: array([0.31154514, 0.402

In [7]:
starts = 3 #number of different starting points to test
trials = 20 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 100, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    for j in range(0, trials):
        res = optimize.basinhopping(CovidEM.Covid_KL, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:



AssertionError: [-0.004670336991580195, -0.0012515829661644167, -0.0021306090809755165, -0.04450822560026256, -0.12563334327330286, -0.10586351496469089, -0.19174082745087243, -0.08921156128548312, -0.16042957266171506, -0.07716531538505204, -0.14260328832027738, 0.09287756686317199, 0.3474696758945371, 0.2521567585090845, 0.9473142721707433, 0.30538990454283965]

In [ ]:
imp.reload(CovidEM) #applies changes made to file

best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081,
            1.727919503, 1.734579037, 3.77747199,  0.71170154,  0.769347828,
            0.104869899, 2.090248998, 1.98923028,  3.373799478]

# Initialize variables
CovidEM.set_prem(prem)
CovidEM.set_cases(cases_dict)
CovidEM.set_countries(countries)

cis11 = CovidEM.CI_calc(best_est, CovidEM.neg_Covid_KL)
print("\n\n\nDone 1")
cis12 = CovidEM.CI_calc(best_est, CovidEM.pos_Covid_KL)
print("\n\n\nDone 2")
#cis21 = CovidEM.CI_calc(CovidEM.transform_parameters(best_est), CovidEM.neg_Covid_KL)
#print("\n\n\nDone 3")
#cis22 = CovidEM.CI_calc(CovidEM.transform_parameters(best_est), CovidEM.pos_Covid_KL)
#print("\n\n\nDone all")

print(cis12)
print(CovidEM.transform_parameters(cis12[:,0]))
print(best_est - cis12[:,0])
print(best_est - cis12[:,1])

In [ ]:
print(best_est)
print(CovidEM.transform_parameters(cis[:,0]))
print(best_est - (cis[:,1]))

In [ ]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          cis[:,0],
          best_est,
          cis[:,1]]
print(best_est - cis[:,0])
print(best_est)
print(best_est - cis[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)


In [ ]:
with open('CovidStratified/Output/CIEstimates.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(ci_header)

    # write multiple rows
    writer.writerows(ci_res)
    
f.close()

In [ ]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
   k, p = params
   return np.exp(k), 1/(1+np.exp(-p))

# Log-Likelihood function for a negative binomial model
def logL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [0, 0]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))

# Define gradient and Hessian
jac = nd.Gradient(logL)
hess = nd.Hessian(logL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, logL, jac, hess,
              disp=True) # the disp argument lets the algorithm print
                         # status messages.
    
print(result.x)
print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
   original_lower[1], original_upper[1]))

In [ ]:
# Example of basin hopping

import numpy as np
from scipy import optimize

# an example function with multiple minima
def f(x): return x.dot(x) + np.sin(np.linalg.norm(x) * np.pi)

# the starting point
x0 = [10., 10.]

# the bounds
xmin = [1., 1.]
xmax = [11., 11.]

# rewrite the bounds in the way required by L-BFGS-B
bounds = [(low, high) for low, high in zip(xmin, xmax)]

# use method L-BFGS-B because the problem is smooth and bounded
minimizer_kwargs = dict(method="L-BFGS-B", bounds=bounds)
res = optimize.basinhopping(f, x0, minimizer_kwargs=minimizer_kwargs)
print(res)

